In [1]:
import jax.numpy as np
from jax import grad
import random
from scipy.optimize import minimize
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import cv2
import glob

ModuleNotFoundError: No module named 'jax'

In [ ]:
M = 5
L = 1
S = 1
dt = 1*1e-4
N = 100

In [2]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [ ]:
def generateRandomCoordinates(pos, L):
    m, n = pos.shape
    for i in range(m):
        for j in range(n):
            pos[i,j] = random.random()*L
            
    return pos


def generateRandomVelocities(vel):
    m, n = vel.shape
    for i in range(m):
        for j in range(n):
            vel[i,j] = np.random.normal(0,S)    # falsche Verteilung
            
    return vel

In [ ]:
def VLJ(r):
    sigma = 0.89#np.power(2,-1/6)
    epsilon = 1
    return 4*epsilon*(np.power(sigma/r,12)-np.power(sigma/r,6))


def Epot(pos):
    pos = pos.reshape((3,M))
    energy = 0
    for i in range(M-1):
        for j in list(range(i+1,M)):
            energy += VLJ(np.linalg.norm(pos[:,i]-pos[:,j]))
            
    return energy


def Ekin(vel):
    energy = 0
    for i in range(M):
        energy += np.linalg.norm(vel[:,i])*np.linalg.norm(vel[:,i])
    energy += 0.5
    
    return energy


In [ ]:
def objective(x):
    return Epot(x)

In [ ]:
pos = np.zeros((3,M))
pos = generateRandomCoordinates(pos, L)
vel = np.zeros((3,M))
vel = generateRandomVelocities(vel)
f = np.zeros((3,M))
print('Epot start: ',Epot(pos.reshape(3*M)))

# plot initial configuration
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(0,L)
ax.set_ylim(0,L)
ax.set_zlim(0,L)
for i in range(M):
    ax.scatter(pos[0,i],pos[1,i],pos[2,i])
plt.show

# setup minimization
b = (0.05,L-0.05)
bnds = list(b for i in range(3*M))
sol = minimize(objective,pos,method='TNC',bounds=bnds)     # not CG method
pos = sol.x.reshape((3,M))
print('Epot new: ', Epot(pos.reshape(3*M)))

# plot new configuration
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(0,L)
ax.set_ylim(0,L)
ax.set_zlim(0,L)
for i in range(M):
    ax.scatter(pos[0,i],pos[1,i],pos[2,i])
plt.show

# velocity calculations
v_average = 0
for i in range(M):
    v_average += vel[:,i]
v_average /= M
for k in range(M):
    vel[:,i] = vel[:,i] - v_average
    
# calculate forces
gradient = grad(Epot)                                # nicht JAX
f = gradient(pos.reshape(3*M)).reshape((3,M))
f

In [ ]:
def calculateNewPosition(pos,vel,f):
    pos_new = np.zeros((3,M))
    for i in range(M):
        pos_new[:,i] = pos[:,i] + vel[:,i]*dt + 0.5*dt*dt*f[:,i]
        if(pos_new[0,i] < 0):
            a = np.ceil(abs(pos_new[0,i])/L)
            pos_new[0,i] += L*a
        if(pos_new[0,i] > L):
            a = np.ceil((pos_new[0,i]-L)/L)
            pos_new[0,i] -= L*a
        if(pos_new[1,i] < 0):
            a = np.ceil(abs(pos_new[1,i])/L)
            pos_new[1,i] += L*a
        if(pos_new[1,i] > L):
            a = np.ceil((pos_new[1,i]-L)/L)
            pos_new[1,i] -= L*a
        if(pos_new[2,i] < 0):
            a = np.ceil(abs(pos_new[2,i])/L)
            pos_new[2,i] += L*a
        if(pos_new[2,i] > L):
            a = np.ceil((pos_new[2,i]-L)/L)
            pos_new[2,i] -= L*a
    return pos_new


def calculateNewVelocity(vel,f,fnew):
    vel_new = np.zeros((3,M))
    for i in range(M):
        vel_new[:,i] = vel[:,i] + 0.5*dt*f[:,i] + 0.5*dt*fnew[:,i]
    return vel_new



def drawSystem(trajectory, name):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
        
    colors = ('blue','red','orange','green','black')
    for n in range(N):
        ax.set_xlim(0,L)
        ax.set_ylim(0,L)
        ax.set_zlim(0,L)
        for i in range(M):
            ax.scatter(trajectory[n][0,i],trajectory[n][1,i],trajectory[n][2,i],c=colors[i])
        call = name+str(n)+'.png'
        plt.savefig(call)
        ax.clear()
        
    
def makeVideo():
    img_array = []
    for filename in glob.glob('*.png'):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

 
    out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
 
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [ ]:
trajectory = [1,]*N
velocetory = [1,]*N
trajectory[0] = pos
velocetory[0] = vel
f_old = gradient(pos.reshape(3*M)).reshape((3,M))
energy = np.zeros(N)
energy[0] = Epot(pos.reshape(3*M))+Ekin(vel)

for i in range(1,N):
    pos = calculateNewPosition(pos,vel,f_old)
    trajectory[i] = pos
    f_new = gradient(pos.reshape(3*M)).reshape((3,M))
    vel = calculateNewVelocity(vel,f_old,f_new)
    velocetory[i] = vel
    f_old = f_new
    energy[i] = Epot(pos.reshape(3*M))+Ekin(vel)
    

In [ ]:
drawSystem(trajectory,'try_')

In [ ]:
makeVideo()

In [ ]:
plt.plot(energy)